<a href="https://colab.research.google.com/github/GabrielPlaza95/T2-Redes-Neurais/blob/main/Trabalho%202%20Redes%20Neurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importação dos dados e arquivos da tarefa

!git clone https://github.com/GabrielPlaza95/T2-Redes-Neurais.git

%cd T2-Redes-Neurais